In [1]:
                           
from pathlib import Path
import os
try :
    PROJECT_DIR =Path (globals ().get ('_dh',[Path .cwd ()])[0 ]).resolve ()
except Exception :
    PROJECT_DIR =Path .cwd ().resolve ()
DATA_DIR =PROJECT_DIR
os .makedirs (DATA_DIR ,exist_ok =True )
print (f"DATA_DIR set to: {DATA_DIR}")


DATA_DIR set to: C:\Users\DELL\OneDrive\Desktop\Final


In [2]:
                           
import pandas as pd ,numpy as np ,random ,re ,os
from datetime import datetime ,timedelta
from difflib import get_close_matches
WEEKS =26
LAMBDA_WEEK =3
ALPHA_SOFTMAX =1.5
OUT_PATH ="engagement_data.csv"
np .random .seed (42 );random .seed (42 )
print ("Parameters set.  Synthetic file will be written to:",OUT_PATH )


Parameters set.  Synthetic file will be written to: engagement_data.csv


In [3]:
                               
DATA_PATH ="/mnt/data"
df_pt =pd .read_csv (str (DATA_DIR /'PT_data.cleaned.csv'))
df_user =pd .read_csv (str (DATA_DIR /'user.csv'))
goal2tags ={
"injury recovery & rehabilitation":["rehabilitation","physiotherapy","functional training"],
"strength building":["strength training","powerlifting"],
"muscle gain":["muscle building","bodybuilding"],
"cardiovascular fitness":["cardio","hiit","crossfit"],
"flexibility & mobility":["flexibility","yoga","pilates"],
"general fitness & wellness":["functional training","yoga","pilates"],
"weight management":["weight loss","hiit","cardio"],
"stress reduction & mindfulness":["yoga","pilates"],
"prenatal & postnatal health":["prenatal fitness"],
"healthy aging / senior wellness":["elderly fitness","functional training"],
"athletic performance & sports rehab":["sports-specific","crossfit"],
"aesthetic & physique goals":["bodybuilding","muscle building"],
}
def speciality_match (spec_string :str ,goal :str )->bool :
    """Exact ▸ synonym ▸ fuzzy (difflib)"""
    toks =[t .strip ().lower ()for t in re .split (r"[,&/]",spec_string )]
    goal_lc =goal .lower ().strip ()
    if goal_lc in toks :
        return True
    if goal_lc in goal2tags and any (tag in toks for tag in goal2tags [goal_lc ]):
        return True
    return bool (get_close_matches (goal_lc ,toks ,n =1 ,cutoff =0.83 ))
print ("Loaded:",len (df_pt ),"trainers  |  ",len (df_user ),"users")


Loaded: 1000 trainers  |   40000 users


In [4]:
                       
df_pt ["q_quality"]=np .random .beta (a =2 ,b =2 ,size =len (df_pt ))
rows ,interaction_id =[],1
today =datetime .today ().replace (hour =0 ,minute =0 ,second =0 ,microsecond =0 )
for _ ,user in df_user .iterrows ():
    uid ,goal =user ["user_id"],user ["health_goal"]
    cand =df_pt [df_pt ["specialities"].apply (lambda s :speciality_match (s ,goal ))]
    if cand .empty :
        continue
    probs =np .exp (ALPHA_SOFTMAX *cand ["q_quality"].values )
    probs /=probs .sum ()
    for week in range (WEEKS ):
        n_int =np .random .poisson (LAMBDA_WEEK )
        for _ in range (n_int ):
            idx =np .random .choice (cand .index ,p =probs )
            trainer_id ,q =cand .at [idx ,"trainer_id"],cand .at [idx ,"q_quality"]
            vc_rate =np .random .beta (1 +3 *q ,1 +3 *(1 -q ))
            adherence =vc_rate *(0.8 +0.2 *np .random .rand ())
            feedback =np .clip (1 +4 *q +np .random .normal (0 ,0.5 ),1 ,5 )
            days_ago =week *7 +np .random .randint (0 ,7 )
            timestamp =today -timedelta (days =days_ago ,
            hours =np .random .randint (0 ,24 ),
            minutes =np .random .randint (0 ,60 ))
            rows .append ({
            "interaction_id":interaction_id ,
            "timestamp":timestamp ,
            "user_id":uid ,
            "trainer_id":trainer_id ,
            "video_completion_rate":round (vc_rate ,3 ),
            "adherence_score":round (adherence ,3 ),
            "feedback_score":round (feedback ,2 )
            })
            interaction_id +=1
df_syn =pd .DataFrame (rows ).sort_values ("timestamp")
df_syn .to_csv (OUT_PATH ,index =False )
print (f"Wrote {len(df_syn):,} synthetic interactions to {OUT_PATH}")


Wrote 3,120,110 synthetic interactions to engagement_data.csv


In [5]:
                                  
df_syn .head ()


,interaction_id,timestamp,user_id,trainer_id,video_completion_rate,adherence_score,feedback_score
1861284,1861285,2025-02-28 00:01:00,23879,585,0.632,0.534,3.85
393904,393905,2025-02-28 00:01:00,5053,73,0.805,0.792,2.72
220602,220603,2025-02-28 00:01:00,2834,475,0.414,0.356,2.94
465058,465059,2025-02-28 00:01:00,5967,834,0.413,0.388,4.21
1896100,1896101,2025-02-28 00:01:00,24327,83,0.635,0.600,3.36
